In [1]:
# !unzip learn_dataset.zip >/dev/null


In [2]:
# !ls /kaggle/input/testssastat/learn_dataset

In [3]:
import pandas as pd
import numpy as np

In [4]:
# !ls learn_dataset/info.csv

In [5]:
df = pd.read_csv('/kaggle/input/testssastat/learn_dataset/info.csv',sep=';')

In [6]:
df.columns

Index(['Тип', 'Статус', 'Время', 'Схема', 'Откуда', 'Куда', 'Кто ответил',
       'Длительность звонка', 'Длительность разговора', 'Время ответа',
       'Оценка', 'ID записи', 'Метка', 'Теги', 'ID заказа звонка',
       'Запись существует', 'Новый клиент', 'Состояние перезвона',
       'Время перезвона', 'Успешный результат', 'Ответственный из CRM'],
      dtype='object')

In [7]:
df

,Тип,Статус,Время,Схема,Откуда,Куда,Кто ответил,Длительность звонка,Длительность разговора,Время ответа,...,ID записи,Метка,Теги,ID заказа звонка,Запись существует,Новый клиент,Состояние перезвона,Время перезвона,Успешный результат,Ответственный из CRM
0,Входящий,Отвечен,18.05.2022 11:33:47,Евген,7965,+7962,205,28,12.0,16,...,1.652863e+09,NaN,NaN,NaN,1,1,NaN,NaN,True,205 Евгения Фатун
1,Входящий,Отвечен,18.05.2022 11:34:56,Федор,7965,+7903,219,31,12.0,19,...,1.652863e+09,NaN,NaN,NaN,1,0,NaN,NaN,False,219 Софья Султанова
2,Входящий,Отвечен,19.05.2022 18:02:22,Татья,7928,+7903,209,56,27.0,29,...,1.652973e+09,NaN,NaN,NaN,1,1,NaN,NaN,True,209 Татьяна Полубоярова
3,Входящий,Отвечен,19.05.2022 18:13:45,Татья,7938,+7903,209,160,148.0,12,...,1.652973e+09,NaN,NaN,NaN,1,0,NaN,NaN,False,209 Татьяна Полубоярова
4,Входящий,Отвечен,19.05.2022 20:34:34,Татья,7905,+7903,209,173,157.0,16,...,1.652982e+09,NaN,NaN,NaN,1,1,NaN,NaN,True,209 Татьяна Полубоярова
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,Исходящий,Отвечен,26.07.2022 15:29:00,Бойцо,202,+7918,+7918,119,102.0,17,...,1.658839e+09,NaN,NaN,NaN,1,0,NaN,NaN,False,202 Анна Бойцова
2830,Входящий,Отвечен,26.07.2022 16:05:15,8800,7967,88003,210,44,37.0,7,...,1.658841e+09,NaN,NaN,NaN,1,1,NaN,NaN,True,221 Анна Суворова
2831,Исходящий,Отвечен,26.07.2022 16:26:21,88612,221,+7918,+7918,60,42.0,18,...,1.658842e+09,NaN,NaN,NaN,1,0,NaN,NaN,False,221 Анна Суворова
2832,Входящий,Отвечен,26.07.2022 16:26:49,8800,7981,envyz,210,41,32.0,9,...,1.658842e+09,NaN,NaN,NaN,1,1,NaN,NaN,True,210 Ирина Подшивалова


In [8]:
df['ID записи']=df['ID записи'].astype('int')

In [9]:
df['ID записи']

0       1652862827
1       1652862896
2       1652972541
3       1652973225
4       1652981674
           ...    
2829    1658838540
2830    1658840715
2831    1658841981
2832    1658842009
2833    1658842180
Name: ID записи, Length: 2834, dtype: int64

In [10]:
!ls learn_dataset/1652862827*

ls: cannot access 'learn_dataset/1652862827*': No such file or directory


In [11]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import os
import re
import pandas as pd
from tqdm import tqdm
from glob import glob
import soundfile as sf
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import os
import re
import pandas as pd
from tqdm import tqdm
import soundfile as sf
import torchaudio
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the Whisper model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3").to(device)


Using device: cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
from tqdm import tqdm
def resample_audio(audio_input, original_sample_rate, target_sample_rate=16000):
    if original_sample_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=target_sample_rate)
        audio_input = resampler(torch.tensor(audio_input).float())
    return audio_input
def process_batch(files, processor, model, device):
    for file_path in tqdm(files):
        if file_path.endswith('.wav'):
            try:
                file_id = int(re.findall('\d+', os.path.basename(file_path))[0])
                # Read the audio file
                audio_input, original_sample_rate = sf.read(file_path)
                # Resample the audio to 16kHz
                audio_input = resample_audio(audio_input, original_sample_rate).numpy()
                # Preprocess the audio
                input_features = processor(audio_input, sampling_rate=16000, return_tensors="pt").input_features
                # Move input to the device
                input_features = input_features.to(device)
                # Generate the transcription
                predicted_ids = model.generate(input_features)
                transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
                df.loc[df['ID записи'] == file_id, 'text_conv'] = transcription
            except Exception as e:
#                 print(f"Error processing {file_path}: {e}")
                df.loc[df['ID записи'] == file_id, 'text_conv'] = None


In [13]:
    audio_directory = "/kaggle/input/testssastat/learn_dataset"
    wav_files = [os.path.join(audio_directory, f) for f in os.listdir(audio_directory) if f.endswith('.wav')]

    # Batch size (number of files to process at a time)
    batch_size = 4

    # Process files in batches with tqdm for progress tracking
   
    process_batch(wav_files, processor, model, device)

    # Optionally, save the DataFrame to a file after processing

  0%|          | 0/2834 [00:00<?, ?it/s]2024-06-22 10:32:51.282160: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 10:32:51.282218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 10:32:51.283648: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English

In [14]:
df['text_conv'].isna().sum()

584

In [15]:
len(df['text_conv'])

2834

In [16]:
df['text_conv']

0        Клиника за один день. Евгения, здравствуйте. ...
1        Алло. Алло. Компания CPUNY, это звонок. Здрав...
2                                                  Hello?
3                                   Татьяна Владимировна.
4        Сейчас на какой звонишь? На мегафон звоню. А ...
                              ...                        
2829     Алло, Виктория Владимировна, здравствуйте. Зд...
2830                                     Звонок от ZUN.ru
2831                                                  Да.
2832                                                 None
2833                                                 None
Name: text_conv, Length: 2834, dtype: object

In [17]:
df.to_csv('submission.csv', index=True)

In [18]:
from IPython.display import FileLink
FileLink('submission.csv') # file

/kaggle/working/submission.csv